In [1]:
import weave
from typing import Any

@weave.op()
def inc(a: int, delta: int = 1) -> int:
    return a + delta

# store x in local dir
x = weave.save(10, "local-num")
print(x)
# store y as a remote artifact - this is stateful and will create
# new versions if you've previously saved this artifact
y = weave.publish(100, "remote-num")
print(y)

get("local-artifact:///tmp/local-artifacts/local-num/acdcdd972ccb68afe0f04a99a639ceb4")
get("wandb-artifact://bbischof-wandb/weave_ops/remote-num:v0")


In [2]:
# increment constant
print(weave.use(inc(1)))
# increment local number - DO NOT SAVE
print(weave.use(inc(x)))
# increment remote number - DO NOT SAVE
print(weave.use(inc(y)))

2
11
101


In [3]:
# this might take a minute since constructing w&b artifacts is a bit slow atm
for i in range(0, 5):
    y = weave.use(inc(y, 1))
    # publish result back to remote-num
    weave.publish(y, "remote-num")

In [4]:
# TODO: this doesn't work yet, it only returns the current version
for version in weave.versions(y):
    print(version)

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/local-artifacts/remote-num'

In [5]:
# we can also publish ops
remote_op = weave.publish(inc)
print(remote_op)

get("wandb-artifact://bbischof-wandb/weave_ops/op-def:v0")


In [7]:
# this cell can be run from a full kernel restart and it should work
import weave
import shutil
from weave.artifacts_local import LOCAL_ARTIFACT_DIR
from weave import uris

try:
    shutil.rmtree(LOCAL_ARTIFACT_DIR)
except FileNotFoundError:
    pass

remote_y = uris.WeaveURI.parse('wandb-artifact://bbischof-wandb/weave_ops/remote-num:v0').to_ref().get()
remote_inc = uris.WeaveURI.parse('wandb-artifact://bbischof-wandb/weave_ops/op-def-op-inc:v0').to_ref().get()

remote_y

CommError: Project bbischof-wandb/weave_ops does not contain artifact: "op-def-op-inc:v0"